In [1]:
import datasets
import json
import os
import numpy as np
import collections
import re
import string
import math

In [2]:
# Change these variables to perform evalution for you task
test_path = os.path.abspath(os.getcwd()).split('gpt3_evaluation_scripts')[0] + 'processed_data/gpt3/completion_4/processed_test.json'
generated_path = os.path.abspath(os.getcwd()).split('gpt3_evaluation_scripts')[0] + 'generated_data_gpt3/experiment_4/generated_answers_macaw.json'
distractor_path = os.path.abspath(os.getcwd()).split('gpt3_evaluation_scripts')[0] + 'generated_data_gpt3/experiment_4/generated_distractors.json'

In [3]:
# Get all the target prompts and completions
test_data = []

for line in open(test_path):
    test_data.append((json.loads(line)))
    
generated_data = []

# Get the generated completions
with open(generated_path) as f:
    generated = json.load(f)

for key in generated:
    generated_data.append(generated[key])
    
distractor_data = []
# Get the generated completions
with open(distractor_path) as f:
    distractors = json.load(f)

for key in generated:
    distractor_data.append(distractors[key])

In [4]:
count = 0
similar_count = 0

predictions = []
gold_references = []

for i in range(len(test_data)):
    count += 1
    
    temp = test_data[i]['prompt']
    completion = temp.split('\n\nAnswer: ')[1].split('\n\n###')[0]
    clean_completion = completion.replace(" ","").replace("~","")
    
    generated_completion = generated_data[i][0].split('= ')[-1]
    clean_generated_completion = generated_completion.replace(" ","")
    
    if clean_completion == clean_generated_completion:
        similar_count += 1
        
    predictions.append(generated_completion)
    gold_references.append(completion)
    
print('Accuracy:', round(similar_count/count * 100, 2), '%')

Accuracy: 75.16 %


In [5]:
# Create result files

with open(os.path.abspath(os.getcwd()).split('gpt3_evaluation_scripts')[0] + 'generated_data_gpt3/experiment_4/' + 'predictions_macaw.txt', 'w') as f:
    for i in range(len(predictions)):
        f.write(predictions[i] + '\n')
        
with open(os.path.abspath(os.getcwd()).split('gpt3_evaluation_scripts')[0] + 'generated_data_gpt3/experiment_4/' + 'ground_truth_macaw.txt', 'w') as f:
    for i in range(len(gold_references)):
        f.write(gold_references[i] + '\n')
        
with open(os.path.abspath(os.getcwd()).split('gpt3_evaluation_scripts')[0] + 'generated_data_gpt3/experiment_4/' + 'automatic_evaluation.txt', 'w') as f:
    f.write('Accuracy: ' + str(round(similar_count/count * 100, 2)) + '%')

In [6]:
# Put the predictions and gold references from distractors in lists

predictions = []
gold_references = []

for i in range(len(test_data)):
    
    completion = test_data[i]['completion']
    clean_completion = completion.split('False answer: ')
    distractor_1 = clean_completion[1].split('\n')[0]
    distractor_2 = clean_completion[2].split('\n')[0]
    distractor_3 = clean_completion[3].split('\n')[0]
    
    gold_references.append([distractor_1, distractor_2, distractor_3])
    
    generated_completion = distractor_data[i]
    clean_generated_completion = generated_completion.split('False answer:')
    generated_distractor_1 = clean_generated_completion[1].split('\n')[0].strip()
    generated_distractor_2 = clean_generated_completion[2].split('\n')[0].strip()
    generated_distractor_3 = clean_generated_completion[3].split('\n')[0].strip()

    predictions.append([generated_distractor_1, generated_distractor_2, generated_distractor_3])

In [7]:
count = 0
with open(os.path.abspath(os.getcwd()).split('gpt3_evaluation_scripts')[0] + 'human_evaluation/experiment4/' + 'exp4_gpt3.txt', 'w') as f:
    for i in range(0,len(test_data),math.floor(len(test_data)/100)):
        f.write('Excel row: ' + str(count+2) + ' Test instance: ' + str(i+1) + '\n\n')
        f.write(test_data[i]['prompt'].split("\n\n###")[0] + '\n\n')
        f.write('Generated: ' + '\n')
        f.write('False answer: ' + predictions[i][0] + '\n')
        f.write('False answer: ' + predictions[i][1] + '\n')
        f.write('False answer: ' + predictions[i][2] + '\n\n')
        f.write('----------------------------------------------------------------------------------------' + '\n\n')
        count+=1
        
        if count == 100:
            break